In [1]:
import oggm 
from oggm import cfg, workflow, utils
from oggm.workflow import execute_entity_task
import geopandas as gpd
import salem, os, netCDF4, logging
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime
log = logging.getLogger(__name__)

In [25]:
cfg.initialize()
cfg.PARAMS['continue_on_error'] = True
cfg.PATHS['climate_file'] = "/home/david/Desktop/CCSM4_midHolocene_data/127ka.nc"
cfg.PARAMS['use_multiprocessing'] = True
PI_path = "/home/david/Desktop/CCSM4_midHolocene_data/pi.nc"
rgi_version = '6'
rgi_region = '17'
cfg.PATHS['working_dir'] = "/home/david/OGGM_WORKING_DIRECTORY/"
path = utils.get_rgi_intersects_region_file(rgi_region, version=rgi_version)
cfg.set_intersects_db(path)

2018-04-28 16:53:07: oggm.cfg: Parameter file: /home/david/oggm/oggm/params.cfg


In [3]:
rgi = gpd.read_file(os.path.join(cfg.PATHS['working_dir'], 'rgi_pif.shp'))
gdirs = oggm.workflow.init_glacier_regions(rgi)

2018-04-28 16:36:44: oggm.workflow: Multiprocessing: using all available processors (N=12)


In [4]:
gdir = gdirs[0]
gdir

<oggm.GlacierDirectory>
  RGI id: RGI60-17.00086
  Region: 17: Southern Andes
  Subregion: 17-01: Patagonia                       
  Glacier type: Glacier
  Terminus type: Land-terminating
  Area: 0.093 km2
  Lon, Lat: (-73.1545, -50.8267)
  Grid (nx, ny): (85, 57)
  Grid (dx, dy): (14.0, -14.0)

In [5]:
fpath = gdir.get_filepath('cesm_data')
with netCDF4.Dataset(fpath, mode='r') as nc:
    time = nc.variables['time']
    time = netCDF4.num2date(time[:], time.units)
time

array([datetime.datetime(1950, 4, 1, 0, 0),
       datetime.datetime(1950, 5, 1, 0, 0),
       datetime.datetime(1950, 6, 1, 0, 0),
       datetime.datetime(1950, 7, 1, 0, 0),
       datetime.datetime(1950, 8, 1, 0, 0),
       datetime.datetime(1950, 9, 1, 0, 0),
       datetime.datetime(1950, 10, 1, 0, 0),
       datetime.datetime(1950, 11, 1, 0, 0),
       datetime.datetime(1950, 12, 1, 0, 0),
       datetime.datetime(1951, 1, 1, 0, 0),
       datetime.datetime(1951, 2, 1, 0, 0),
       datetime.datetime(1951, 3, 1, 0, 0)], dtype=object)

In [26]:
#Put this in the function def (see process_cesm_data)
filesuffix=''

if not (('climate_file' in cfg.PATHS) and
        os.path.exists(cfg.PATHS['climate_file'])):
    raise IOError('Custom climate file not found')

#open dataset for precp use
fpath = cfg.PATHS['climate_file']
xr_ccsm = xr.open_dataset(fpath, decode_times=False)
#open dataset for tmp use
xr_ccsm_ts = xr.open_dataset(fpath)
#repeating for pi
xr_pi = xr.open_dataset(PI_path, decode_times=False)

# selecting location
lon = gdir.cenlon
lat = gdir.cenlat

#Setting the longitude to a 0-360 grid [I think...] "CESM files are in 0-360"
if lon <= 0:
    lon += 360

#"take the closest"
#"TODO: consider GCM interpolation?"
precp = xr_ccsm.PRECC.sel(lat=lat, lon=lon, method='nearest') + xr_ccsm.PRECL.sel(lat=lat, lon=lon, method='nearest')
temp = xr_ccsm_ts.TS.sel(lat=lat, lon=lon, method='nearest')
precp_pi = xr_pi.PRECC.sel(lat=lat, lon=lon, method='nearest') + xr_pi.PRECL.sel(lat=lat, lon=lon, method='nearest')
temp_pi = xr_pi.TS.sel(lat=lat, lon=lon, method='nearest')

#convert temp from K to C
temp = temp - 273.15
temp_pi = temp_pi - 273.15

#Take anomaly for C|CSM data (with preindustrial control)
for i in range(12):
    temp.values[i] = temp.values[i] - temp_pi.values[i]
for i in range(12):
    precp.values[i] = precp.values[i] - precp_pi.values[i]

In [7]:
#selecting & formatting values for y0 & y1
y_0 = pd.to_datetime(str(temp.time.values[0]))
y0= int(y_0.strftime('%Y'))
y_1 = pd.to_datetime(str(temp.time.values[-1]))
y1 = int(y_1.strftime('%Y'))

In [8]:
#reformatting the time element of the data array to a datetime64.
time = pd.period_range('{}-02'.format(y0), '{}01'.format(y1), freq='M')
temp['time'], precp['time'] = time, time

In [9]:
# Workaround for https://github.com/pydata/xarray/issues/1565
temp['month'] = ('time', time.month)
precp['month'] = ('time', time.month)
temp['year'] = ('time', time.year)
temp['year'] = ('time', time.year)
ny, r = divmod(len(temp.time), 12)
assert r == 0


In [10]:
#Convert m s-1 to mm mth-1 (for precp)
ndays = np.tile(cfg.DAYS_IN_MONTH, y1-y0)
precp = precp * ndays * (60 * 60 * 24 * 1000)

#"Get CRU to apply the anomaly to"
fpath = gdir.get_filepath('climate_monthly')
ds_cru = xr.open_dataset(fpath)

#"Here we assume the gradient is a monthly average"
ts_grad = np.tile(ds_cru.grad[0:12], ny)

In [11]:
#"Add the climate anomaly to CRU clim"
dscru = ds_cru.sel(time=slice('1961', '1990'))
#"for temp"
loc_tmp = dscru.temp.groupby('time.month').mean()
loc_tmp = xr.concat([loc_tmp[1:], loc_tmp[:-11]], dim='month')
ts_tmp = temp #+ loc_tmp
#ts_tmp[:] = [(i - 5) for i in temp]
for j in range(12):
    ts_tmp.values[j] = ts_tmp[j].values + loc_tmp.values[j]
ts_tmp = xr.concat([ts_tmp[2:], ts_tmp[:-10]], dim='time')

In [12]:
#"for prcp"
loc_pre = dscru.prcp.groupby('time.month').mean()
loc_pre = xr.concat([loc_pre[1:], loc_pre[:-11]], dim='month')
ts_pre = precp
for j in range(12):
    ts_pre.values[j] = ts_pre[j].values + loc_pre.values[j]
ts_pre = xr.concat([ts_pre[2:], ts_pre[:-10]], dim='time')

In [13]:
fpath = cfg.PATHS['climate_file']
dsindex = salem.GeoNetcdf(fpath, monthbegin=True)
time1 = dsindex.variables['time']

In [14]:
time2 = [711475., 711505.,711536., 711566.,711597., 711628.,711658., 711689.,711719., 711750.,711781., 711809.]
time2 = netCDF4.num2date(time2, time1.units, calendar='noleap')

In [15]:
assert np.all(np.isfinite(ts_pre.values))
assert np.all(np.isfinite(ts_tmp.values))
assert np.all(np.isfinite(ts_grad))

#"back to -180 - 180"
loc_lon = precp.lon if precp.lon <= 180 else precp.lon - 360

#write to netcdf
gdir.write_monthly_climate_file(time2, ts_pre.values, ts_tmp.values, ts_grad, float(dscru.ref_hgt), loc_lon, precp.lat.values, time_unit=time1.units, file_name='cesm_data', filesuffix=filesuffix)

#dsindex._nc.close()
xr_ccsm.close()
xr_ccsm_ts.close()
xr_pi.close()
ds_cru.close()

#logger message
log.info("%s", gdir.rgi_id)

2018-04-28 16:37:45: __main__: RGI60-17.00086


## CCSM DATA PROCESSING -- > DO NOT GOOF WITH

In [1]:
import geopandas as gpd
import salem, os, netCDF4, logging
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime

def process_ccsm_data(gdir):
    
    """
        First attempt at a method to process the CCSM data into temperature/precip anomalies.

    """
 
    #Put this in the function def (see process_cesm_data)
    filesuffix=''

    if not (('climate_file' in cfg.PATHS) and
            os.path.exists(cfg.PATHS['climate_file'])):
        raise IOError('Custom climate file not found')

    #open dataset for precp use
    fpath = cfg.PATHS['climate_file']
    xr_ccsm = xr.open_dataset(fpath, decode_times=False)
    #open dataset for tmp use
    xr_ccsm_ts = xr.open_dataset(fpath, decode_times=False)
    #repeating for pi
    xr_pi = xr.open_dataset(PI_path, decode_times=False)

    # selecting location
    lon = gdir.cenlon
    lat = gdir.cenlat

    #Setting the longitude to a 0-360 grid [I think...] "CESM files are in 0-360"
    if lon <= 0:
        lon += 360 

    #"take the closest"
    #"TODO: consider GCM interpolation?"
    precp = xr_ccsm.PRECC.sel(lat=lat, lon=lon, method='nearest') + xr_ccsm.PRECL.sel(lat=lat, lon=lon, method='nearest')
    temp = xr_ccsm_ts.TS.sel(lat=lat, lon=lon, method='nearest')
    precp_pi = xr_pi.PRECC.sel(lat=lat, lon=lon, method='nearest') + xr_pi.PRECL.sel(lat=lat, lon=lon, method='nearest')
    temp_pi = xr_pi.TS.sel(lat=lat, lon=lon, method='nearest')

    #convert temp from K to C
    temp = temp - 273.15
    temp_pi = temp_pi - 273.15

   #Take anomaly for C|CSM data (with preindustrial control)
    for i in range(12):
        temp.values[i] = temp.values[i] - temp_pi.values[i]
    for i in range(12):
        precp.values[i] = precp.values[i] - precp_pi.values[i]

    #selecting & formatting values for y0 & y1
    y_0 = pd.to_datetime(str(temp.time.values[0]))
    y0= int(y_0.strftime('%Y'))
    y_1 = pd.to_datetime(str(temp.time.values[-1]))
    y1 = int(y_1.strftime('%Y'))

    #reformatting the time element of the data array to a datetime64.
    time = pd.period_range('{}-02'.format(y0), '{}01'.format(y1), freq='M')
    temp['time'], precp['time'] = time, time

    temp['month'] = ('time', time.month)
    precp['month'] = ('time', time.month)
    temp['year'] = ('time', time.year)
    temp['year'] = ('time', time.year)
    ny, r = divmod(len(temp.time), 12)
    assert r == 0

    #Convert m s-1 to mm mth-1 (for precp)
    ndays = np.tile(cfg.DAYS_IN_MONTH, y1-y0)
    precp = precp * ndays * (60 * 60 * 24 * 1000)

    #"Get CRU to apply the anomaly to"
    fpath = gdir.get_filepath('climate_monthly')
    ds_cru = xr.open_dataset(fpath)

    #"Here we assume the gradient is a monthly average"
    ts_grad = np.tile(ds_cru.grad[0:12], ny)

    #"Add the climate anomaly to CRU clim"
    dscru = ds_cru.sel(time=slice('1961', '1990'))
    #"for temp"
    loc_tmp = dscru.temp.groupby('time.month').mean()
    loc_tmp = xr.concat([loc_tmp[1:], loc_tmp[:-11]], dim='month')
    ts_tmp = temp #+ loc_tmp
    for j in range(12):
        ts_tmp.values[j] = ts_tmp[j].values + loc_tmp.values[j]
    ts_tmp = xr.concat([ts_tmp[2:], ts_tmp[:-10]], dim='time')
    #"for prcp"
    loc_pre = dscru.prcp.groupby('time.month').mean()
    loc_pre = xr.concat([loc_pre[1:], loc_pre[:-11]], dim='month')
    ts_pre = precp
    for j in range(12):
        ts_pre.values[j] = ts_pre[j].values + loc_pre.values[j]
    ts_pre = xr.concat([ts_pre[2:], ts_pre[:-10]], dim='time')

    #"load dates into right format to save"
    fpath = cfg.PATHS['climate_file']
    dsindex = salem.GeoNetcdf(fpath, monthbegin=True)
    time1 = dsindex.variables['time']

    assert np.all(np.isfinite(ts_pre.values))
    assert np.all(np.isfinite(ts_tmp.values))
    assert np.all(np.isfinite(ts_grad))

    time2 = [711475., 711505.,711536., 711566.,711597., 711628.,711658., 711689.,711719., 711750.,711781., 711809.]
    time2 = netCDF4.num2date(time2, time1.units, calendar='noleap')

    #"back to -180 - 180"
    loc_lon = precp.lon if precp.lon <= 180 else precp.lon - 360

    #write to netcdf
    gdir.write_monthly_climate_file(time2, ts_pre.values, ts_tmp.values, ts_grad, float(dscru.ref_hgt), loc_lon, precp.lat.values, time_unit=time1.units, file_name='cesm_data', filesuffix=filesuffix)

    #dsindex._nc.close()
    xr_ccsm.close()
    xr_ccsm_ts.close()
    xr_pi.close()
    ds_cru.close()

    #logger message
    log.info("%s", gdir.rgi_id)
    
def process_ccsm_gdirs(gdirs):
    for i in gdirs:
        new_process_ccsm_data(i)